## Before starting this tutorial
- Install DFTTK: https://github.com/PhasesResearchLab/dfttk
- Install YPHON: https://github.com/PhasesResearchLab/YPHON 
- Follow the POTCAR setup for pymatgen: https://pymatgen.org/installation.html#potcar-setup

## Import neccessary libraries

In [ ]:
# Enable automatic reloading of modules
%reload_ext autoreload
%autoreload 2

# Standard Library Imports
import os
import subprocess
from natsort import natsorted

# Third-Party Library Imports
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# DFTTK Imports
from dfttk.atat.caller import Icamag
from dfttk.configuration import Configuration, plot_multiple_ev
from dfttk.plotly_format import plot_format

## Convergence tests

Create a folder to run the volume relaxation. We will use the relaxed structure for our convergence tests and to generate the magnetic configurations.

In [ ]:
path = "conv_test/volume_relax"
os.makedirs(path, exist_ok=True)

Copy the `POSCAR` file to the volume relaxation folder.

In [ ]:
subprocess.run(["cp", "POSCAR", path])

Create and configure the Configuration object. 

In [ ]:
vasp_cmd = ["mpirun", "/opt/packages/VASP/VASP6/6.4.3/ONEAPI/vasp_std"]
config_0 = Configuration(path, "config_0", vasp_cmd=vasp_cmd)

Configure the `job script` for volume relaxation. This example is based on the Bridges-2 supercomputer at the Pittsburgh Supercomputing Center (PSC).

In [ ]:
# Write the job script to the config_0.path directory
job_script = """#!/bin/bash
#SBATCH --job-name=Fe3Pt
#SBATCH -A dmr190011p
#SBATCH -p RM
#SBATCH -N 1
#SBATCH --ntasks-per-node=64
#SBATCH -t 1:00:00
#SBATCH -o vasp.out
#SBATCH -e vasp.err

ulimit -s unlimited
export OMP_NUM_THREADS=1
echo "SLURM_NTASKS: " $SLURM_NTASKS
module purge
module load intel-oneapi
module load hdf5/1.12.0-intel20.4
module load anaconda3
conda activate dfttk
module load VASP/6.4.3-intel
python run_dfttk.py
echo "Current SLURM JobID: $SLURM_JOB_ID"
sacct --job=$SLURM_JOB_ID
"""

with open(os.path.join(config_0.path, "job.sh"), "w") as f:
    f.write(job_script)

🚀 Run the volume relaxation.

In [ ]:
config_0.run_volume_relax(material_type="metal", magmom_fm=True)

⏳ After the job has finished, copy the relaxed `CONTCAR` file to the directory above. We will use this for the convergence tests.

In [ ]:
subprocess.run(["cp", "CONTCAR", "../POSCAR"], cwd=path)

Update the configuration path for the convergence tests.

In [ ]:
config_0.path = "conv_test"

Write the `job script` for the convergence tests.

In [ ]:
# Write the job script to the config_0.path directory
job_script = """#!/bin/bash
#SBATCH --job-name=Fe3Pt
#SBATCH -A dmr190011p
#SBATCH -p RM
#SBATCH -N 1
#SBATCH --ntasks-per-node=64
#SBATCH -t 1:00:00
#SBATCH -o vasp.out
#SBATCH -e vasp.err

ulimit -s unlimited
export OMP_NUM_THREADS=1
echo "SLURM_NTASKS: " $SLURM_NTASKS
module purge
module load intel-oneapi
module load hdf5/1.12.0-intel20.4
module load anaconda3
conda activate dfttk
module load VASP/6.4.3-intel
python run_dfttk.py
echo "Current SLURM JobID: $SLURM_JOB_ID"
sacct --job=$SLURM_JOB_ID
"""

with open(os.path.join(config_0.path, "job.sh"), "w") as f:
    f.write(job_script)

🚀 Run the convergence tests.

In [ ]:
config_0.run_conv_test(magmom_fm=True)

⏳ After the jobs have finished, analyze the convergence of the total energy with respect to the cutoff energy (ENCUT).

In [ ]:
encut_conv_df, fig = config_0.analyze_encut_conv()
encut_conv_df

Analyze the convergence of the total energy with respect to the k-point mesh per reciprocal atom (KPPA).

In [ ]:
kpoints_conv_df, fig = config_0.analyze_kpoints_conv()
kpoints_conv_df

To be conservative, we use `ENCUT = 520 eV` and `kppa = 8112`.

## Generate magnetic spin configurations

Create a folder called configs.

In [ ]:
configs_path = "configs" 
os.makedirs(configs_path, exist_ok=True)

We will copy the `POSCAR` file we used for the convergence tests to the configs folder to generate all the possible unique magnetic spin configurations.

In [ ]:
subprocess.run(["cp", "conv_test/POSCAR", os.path.join(configs_path, "POSCAR")], cwd=".")

Initialize the Icamag object with the configuration path.

In [ ]:
icamag = Icamag(configs_path)

Generate all the unique magnetic spin configurations and write the `POSCAR` files to the config_* folders.

In [ ]:
magnetic_sites = {"Fe": ["Fe+5", "Fe-5"]}
icamag.gen_spin_configs(magnetic_sites=magnetic_sites)

Summarize all unique magnetic spin configurations in spin_configs and collect all POSCAR objects in poscar_object_list.

In [ ]:
spin_configs, poscar_object_list = icamag.parse_spin_configs()
spin_configs

Get the multiplicities of the configurations.

In [ ]:
multiplicity = icamag.get_multiplicity()

## Energy-volume curves

Define the path to the configs folder and generate the config_names.

In [ ]:
configs_path = "configs" 
config_names = [f for f in os.listdir(configs_path) if os.path.isdir(os.path.join(configs_path, f)) and f.startswith("config_")]
config_names = natsorted(config_names)

Create the Configuration objects.

In [ ]:
vasp_cmd = ["mpirun", "/opt/packages/VASP/VASP6/6.4.3/ONEAPI/vasp_std"]
config_objects = {name: Configuration(os.path.join(configs_path, name), name, vasp_cmd, "FCC Fe3Pt 12-atom supercell", multiplicity[i]) for i, name in enumerate(config_names)}

Here, we are just going to deal with three lowest energy configurations. 

In [ ]:
selected_indices = [0, 28, 22]
selected_config_names = [config_names[i] for i in selected_indices]

Write the `job script` for the energy-volume curve jobs.

In [ ]:
for config_name in selected_config_names:
    job_script = """#!/bin/bash
    #SBATCH --job-name=Fe3Pt
    #SBATCH -A dmr190011p
    #SBATCH -p RM-shared
    #SBATCH -N 1
    #SBATCH --ntasks-per-node=64
    #SBATCH -t 24:00:00
    #SBATCH -o vasp.out
    #SBATCH -e vasp.err

    ulimit -s unlimited
    export OMP_NUM_THREADS=1
    echo "SLURM_NTASKS: " $SLURM_NTASKS
    module purge
    module load intel-oneapi
    module load hdf5/1.12.0-intel20.4
    module load anaconda3
    conda activate dfttk
    module load VASP/6.4.3-intel
    python run_dfttk.py
    echo "Current SLURM JobID: $SLURM_JOB_ID"
    sacct --job=$SLURM_JOB_ID
    """

    # Write the job script to the config.path directory
    config = config_objects[config_name]
    with open(os.path.join(config.path, "job.sh"), "w") as f:
        f.write(job_script)

Configure the energy-volume settings.

In [ ]:
material_type = "metal"
volumes = [172, 169, 166, 163, 160, 157, 154, 151, 148, 145, 142, 139]
encut = 520
kppa = 8800

for selected_index, selected_config_name in zip(selected_indices, selected_config_names):
    config = config_objects[selected_config_name]
    other_settings = poscar_object_list[selected_index].structure.site_properties
    other_settings.update({"ISPIN": 2, "LORBIT": 11, "ALGO": "All", "NCORE": 8, "LCHARG": True, "NELM": 300})
    config.ev_curve_settings(material_type=material_type, volumes=volumes, encut=encut, kppa=kppa, other_settings=other_settings, copy_magmom=True)

🚀 Run the energy-volume curve calculations for the chosen volumes.

In [ ]:
for config_name in selected_config_names:
    config = config_objects[config_name]
    config.run_ev_curve()

⏳ After the jobs have finished, process the results of the energy-volume curves.

In [ ]:
for config_name in selected_config_names:
    config_objects[config_name].process_ev_curve(collect_mag_data=True)

Plot the energy-volume curve for a single configuration.

In [ ]:
config_objects["config_0"].ev_curve.plot()

Plot the energy-volume curves for multiple configurations.

In [ ]:
config_indices = [0, 28, 22]
config_names = [f"config_{i}" for i in config_indices]

fig = plot_multiple_ev(config_objects, config_names)
fig.update_layout(legend=dict(traceorder='reversed'))
fig.show()

Examine the EOS parameters.

In [ ]:
config_indices = [0, 28, 22]

data = []
for i in config_indices:
    eos_parameters_copy = config_objects[f"config_{i}"].ev_curve.eos_parameters
    eos_parameters_copy['config_index'] = i
    data.append(eos_parameters_copy)

df = pd.DataFrame(data)
columns_order = ['config_index', 'eos_name', 'V0', 'E0', 'B', 'BP', 'B2P']
df = df[columns_order]
df = df.sort_values(by='E0')
config_index_list = df['config_index'].tolist()
df

## Debye-Grüneisen model

Calculate the vibrational contribution to the Helmholtz energy, entropy, and heat capacity using the Debye model.

In [ ]:
config_names = ["config_0", "config_28", "config_22"]
volumes = np.linspace(0.98*139, 1.02*172, 1000)
temperatures = np.arange(0, 1010, 10)

for name in config_names:
    config_objects[name].process_debye(volumes=volumes, temperatures=temperatures, scaling_factor=0.617, gruneisen_x=2/3)

Plot the Debye properties.

In [ ]:
# Plot helmholtz_energy, entropy, or heat_capacity
fig_debye_t, fig_debye_v = config_objects["config_0"].debye.plot("helmholtz_energy")
fig_debye_t.show()
fig_debye_v.show()

## Phonons

Re-write the `job script` for the phonon calculations if needed.

In [ ]:
for config_name in selected_config_names:
    job_script = """#!/bin/bash
    #SBATCH --job-name=Fe3Pt
    #SBATCH -A dmr190011p
    #SBATCH -p RM-shared
    #SBATCH -N 1
    #SBATCH --ntasks-per-node=64
    #SBATCH -t 24:00:00
    #SBATCH -o vasp.out
    #SBATCH -e vasp.err

    ulimit -s unlimited
    export OMP_NUM_THREADS=1
    echo "SLURM_NTASKS: " $SLURM_NTASKS
    module purge
    module load intel-oneapi
    module load hdf5/1.12.0-intel20.4
    module load anaconda3
    conda activate dfttk
    module load VASP/6.4.3-intel
    python run_dfttk.py
    echo "Current SLURM JobID: $SLURM_JOB_ID"
    sacct --job=$SLURM_JOB_ID
    """

    # Write the job script to the config.path directory
    config = config_objects[config_name]
    with open(os.path.join(config.path, "job.sh"), "w") as f:
        f.write(job_script)

Configure the phonon settings.

In [ ]:
config_names = ["config_0", "config_28", "config_22"]
phonon_volumes = [166, 163, 160, 157, 154, 151, 148]
kppa = 8800
scaling_matrix = ((2, 0, 0), (0, 2, 0), (0, 0, 1))
relax = True

for name in config_names:
    config = config_objects[name]
    config.phonons_settings(phonon_volumes=phonon_volumes, kppa=kppa, scaling_matrix=scaling_matrix, relax=relax)

🚀 Run the phonon calculations in parallel.

In [ ]:
for name in config_names:
    config_objects[name].run_phonons()

⏳ After the jobs have finished, generate the phonon DOS using YPHON in each phonon_folder and store all the results in YPHON_results.

In [ ]:
config_names = ["config_0", "config_22", "config_28"]
for name in config_names:
    config_objects[name].generate_phonon_dos()

Using the phonon DOS, calculate the harmonic properties.

In [ ]:
config_names = ["config_0", "config_22", "config_28"]
number_of_atoms = 12
volumes_fit = np.linspace(0.98*139, 1.02*172, 1000)
temperatures = np.arange(0, 1010, 10)
for name in config_names:
    config_objects[name].process_phonons(number_of_atoms=number_of_atoms, volumes_fit=volumes_fit, temperatures=temperatures)

Plot both the original and scaled phonon DOS. The scaled phonon DOS is adjusted to the number of atoms, N, that you specify. YPHON scales the area under the phonon DOS curve to 3N.

In [ ]:
config_objects["config_0"].phonons.plot_scaled_dos(number_of_atoms=12)

Plot the scaled phonon DOS for multiple volumes together.

In [ ]:
config_objects["config_0"].phonons.plot_multiple_dos()

Plot the harmonic properties.

In [ ]:
# Plot helmholtz_energy, entropy, or heat_capacity
fig_phonons_t, fig_phonons_v = config_objects["config_0"].phonons.plot_harmonic("helmholtz_energy")
fig_phonons_t.show()
fig_phonons_v.show()

## Thermal electronic contribution

Re-write the `job script` for the thermal electronic calculations if needed.

In [ ]:
for config_name in selected_config_names:
    job_script = """#!/bin/bash
    #SBATCH --job-name=Fe3Pt
    #SBATCH -A dmr190011p
    #SBATCH -p RM-shared
    #SBATCH -N 1
    #SBATCH --ntasks-per-node=64
    #SBATCH -t 24:00:00
    #SBATCH -o vasp.out
    #SBATCH -e vasp.err

    ulimit -s unlimited
    export OMP_NUM_THREADS=1
    echo "SLURM_NTASKS: " $SLURM_NTASKS
    module purge
    module load intel-oneapi
    module load hdf5/1.12.0-intel20.4
    module load anaconda3
    conda activate dfttk
    module load VASP/6.4.3-intel
    python run_dfttk.py
    echo "Current SLURM JobID: $SLURM_JOB_ID"
    sacct --job=$SLURM_JOB_ID
    """

    # Write the job script to the config.path directory
    config = config_objects[config_name]
    with open(os.path.join(config.path, "job.sh"), "w") as f:
        f.write(job_script)

Configure the thermal electronic settings.

In [ ]:
config_names = ["config_0", "config_28", "config_22"]
volumes = [166, 163, 160, 157, 154, 151, 148]
kppa = 8800
scaling_matrix = ((1, 0, 0), (0, 1, 0), (0, 0, 1))

for name in config_names:
    config = config_objects[name]
    config.thermal_electronic_settings(volumes=volumes, kppa=kppa, scaling_matrix=scaling_matrix)

🚀 Run the thermal electronic jobs in parallel.

In [ ]:
for name in config_names:
    config = config_objects[name]
    config.run_thermal_electronic()

⏳ After the jobs have finished, calculate the thermal electronic properties.

In [ ]:
config_names = ["config_0", "config_28", "config_22"]
volumes_fit = np.linspace(0.98*139, 1.02*172, 1000)
temperatures = np.arange(0, 1010, 10)

for name in config_names:
    config = config_objects[name]
    config.process_thermal_electronic(volumes_fit=volumes_fit, temperatures=temperatures)

Plot the thermal electronic properties.

In [ ]:
# Plot helmholtz_energy, entropy, or heat_capacity
fig_thermal_electronic_t, fig_thermal_electronic_v = config_objects["config_0"].thermal_electronic.plot("helmholtz_energy")
fig_thermal_electronic_t.show()
fig_thermal_electronic_v.update_xaxes(range=[153, 168])
fig_thermal_electronic_v.show()

## Quasiharmonic approximation

Calculate the quasi-harmonic properties using multiple methods: debye, debye_thermal_electronic, phonons, and phonons_thermal_electronic.

In [ ]:
selected_config_names = ["config_0", "config_28", "config_22"]

for config_name in selected_config_names:
    config = config_objects[config_name]
    config.process_qha("debye", P = 0)
    config.process_qha("debye_thermal_electronic", P = 0)
    config.process_qha("phonons", P = 0)
    config.process_qha("phonons_thermal_electronic", P = 0)

Plot the QHA properties.

In [ ]:
# Plot helmholtz_energy_pv, volume, CTE, LCTE, entropy, heat_capacity, enthalpy, bulk_modulus, gibbs_energy
fig_qha = config_objects["config_0"].qha.plot("phonons", P = 0, plot_type="LCTE")
fig_qha.show()

## Comparison with Experiments

In [ ]:
def add_trace(fig, x, y, mode, name, color, dash=None, symbol=None):
    """Helper function to add a trace to the figure."""
    trace = go.Scatter(
        x=x,
        y=y,
        mode=mode,
        name=name,
        line=dict(color=color, dash=dash) if dash else dict(color=color),
        marker=dict(symbol=symbol, color=color) if symbol else None,
    )
    fig.add_trace(trace)

LCTE vs. T

In [ ]:
# Initialize figure
fig = go.Figure()
temperatures = config_objects["config_0"].qha.temperatures

# Experimental data references and scaling
experimental_data = {
    "Sumiyama Fe<sub>72</sub>Pt<sub>28</sub>": {
        "file": "Sumiyama_Fe72Pt28.csv",
        "scale_temp": 900,
        "scale_lcte": lambda x: x * 20 - 5,
        "color": "black",
        "symbol": "circle-open",
    },
    "Sumiyama Fe<sub>3</sub>Pt": {
        "file": "Sumiyama_Fe3Pt.csv",
        "scale_temp": 900,
        "scale_lcte": lambda x: x * 20 - 5,
        "color": "black",
        "symbol": "circle",
    },
    "Rellinghaus Fe<sub>72</sub>Pt<sub>28</sub>": {
        "file": "Rellinghaus_Fe72Pt28.csv",
        "scale_temp": 900,
        "scale_lcte": lambda x: x * 20 - 5,
        "color": "black",
        "symbol": "diamond-open",
    },
}

# Add experimental data
for name, props in experimental_data.items():
    data = pd.read_csv(props["file"], header=None, names=["Temperature", "LCTE"])
    data["Temperature"] *= props["scale_temp"]
    data["LCTE"] = props["scale_lcte"](data["LCTE"])
    add_trace(fig, data["Temperature"], data["LCTE"], mode="markers", name=name, color=props["color"], symbol=props["symbol"])

# Data from calculations for multiple configurations
configs = {
    "SF28": {
        "data": config_objects["config_28"].qha.methods["debye"]['0.00_GPa']['LCTE'],
        "color": "#EF553B",
    },
    "SF22": {
        "data": config_objects["config_22"].qha.methods["debye"]['0.00_GPa']['LCTE'],
        "color": "#00CC96",
    },
    "FM": {
        "data": config_objects["config_0"].qha.methods["debye"]['0.00_GPa']['LCTE'],
        "color": "#636EFA",
    },
}

# Add traces for each configuration
for name, props in configs.items():
    add_trace(fig, temperatures, props["data"], mode="lines", name=name, color=props["color"])

# Format and display the plot
plot_format(fig, xtitle="Temperature (K)", ytitle="LCTE (10<sup>-6</sup> K<sup>-1</sup>)", width=750, height=600)
fig.show()

## MongoDB

In [ ]:
# Define configuration names and their corresponding comments
config_comments = {
    "config_0": "FM",      
    "config_28": "SF28",   
    "config_22": "SF22",   
}

# Add metadata with the appropriate comment for each configuration
for config_name, comment in config_comments.items():
    config_objects[config_name].add_metadata(comment=f"{comment} FCC Fe3Pt 12-atom supercell")

In [ ]:
# Enter the details of your MongoDB database
connection_string = "enter_connection_string"
db_name = "enter_db_name"
collection_name = "enter_collection_name"

for config_name in config_names:
    config_objects[config_name].to_mongodb(connection_string, db_name, collection_name)